In [ ]:
import email
import imaplib
import re
from datetime import datetime
from io import BytesIO

import camelot
import pandas as pd
from bs4 import BeautifulSoup

In [2]:
prev = 0


def pretty(label: str, completed: float, total: int, length: int = 30) -> None:
    global prev
    print(
        " " * (prev * 2)
        + f"\r{label} ["
        + "=" * int(completed / total * length)
        + "-" * int((total - completed) / total * length)
        + f"] {int(completed / total * 100)}%",
        end="",
    )
    prev = length


def prettyPrintReplace(label: str) -> None:
    global prev
    label = str(label)
    print(" " * (prev * 2) + f"\r{label}", end="")
    prev = len(label)

In [ ]:
# Login credentials
IMAP_HOST = "imap.gmail.com"
EMAIL_USER = "email"
EMAIL_PASS = "random"

In [4]:
subject_to_search = "UPGRADE MANDI - Hyperpure"

In [ ]:
# Connect and login
mail = imaplib.IMAP4_SSL(IMAP_HOST)
mail.login(EMAIL_USER, EMAIL_PASS)
mail.select("inbox")

dt = "09-Jun-2025"

dateFilter = f'ON "{dt}" '

if True:
    dateFilter = ""

# Search emails by subject
statusRaw, messagesRaw = mail.search(
    None, f'({dateFilter}SUBJECT "{subject_to_search}")'
)

In [ ]:
grnPrDF = pd.DataFrame({"Cut/Veg": [], "Date": [], "GRN": [], "PO Amount": []})

status = statusRaw
messages = messagesRaw

if status == "OK":
    for num in messages[0].split():
        typ, data = mail.fetch(num, "(RFC822)")
        msg = email.message_from_bytes(data[0][1])

        processedDOMTree = ""
        cutVeg = {"value": None}

        subject = re.sub("\s+", " ", msg["subject"])
        indexPO = re.search(r"CPCMH\d{2}-PO-\d{7}", subject)

        if indexPO:
            indexPO = indexPO.group()
        else:
            raise Exception("Can't find PO number from mail.")

        if indexPO not in grnPrDF.index:
            grnPrDF.loc[indexPO] = ["", "", 0.0, 0.0]

        for part in msg.walk():
            content_type = part.get_content_type()
            content_disposition = str(part.get("Content-Disposition") or "")

            if part.get_content_maintype() == "multipart":
                continue
            elif content_type == "application/pdf":
                cutVeg = "VEG"
                pdfFileData = part.get_payload(decode=True)
                pdfFileStream = BytesIO(pdfFileData)
                try:
                    pdfParserObject = camelot.read_pdf(pdfFileStream)
                    if len(pdfParserObject[0].df) >= 5:
                        findCut = pdfParserObject[0].df.iloc[5][1]
                        findMatches = re.findall(r".*cut.*", findCut, re.IGNORECASE)
                        if len(findMatches) != 0:
                            cutVeg = "CUT-VEG"
                except:
                    cutVeg = "Error"
                grnPrDF.loc[indexPO, "Cut/Veg"] = cutVeg
            elif content_type == "text/html":
                rawHTMLText = part.get_payload(decode=True).decode()

                processedDOMTree = BeautifulSoup(rawHTMLText, "html.parser")

                amountDate = datetime.strptime(
                    processedDOMTree.find_all("table")[1]
                    .find_all("tr")[3]
                    .find_all("td")[1]
                    .find("span")
                    .text,
                    "%d %b %Y",
                ).strftime("%d-%m-%Y")

                amount = float(
                    processedDOMTree.find_all("table")[1]
                    .find_all("tr")[4]
                    .find_all("td")[1]
                    .find("span")
                    .text
                )

                isGrnAmount = (
                    "UPGRADE MANDI - Hyperpure GRN against PO Number" in subject
                )
                isTotalAmount = "UPGRADE MANDI - Hyperpure PO Number" in subject

                if isGrnAmount:
                    grnPrDF.loc[indexPO, "GRN"] += amount
                if isTotalAmount:
                    grnPrDF.loc[indexPO, "PO Amount"] += amount
                grnPrDF.loc[indexPO, "Date"] = amountDate

        print(
            f"Count: {len(grnPrDF)}",
            f"Date: {amountDate}",
            f"Index PO: {indexPO}",
            f'GRN total: {grnPrDF.iloc[-1]["GRN"]}',
            f'Total: {grnPrDF.iloc[-1]["PO Amount"]}',
            f'Cut: {grnPrDF.iloc[-1]["Cut/Veg"]}',
            sep="\n",
            end="\n\n\n",
        )

In [ ]:
grnPrDF

In [12]:
grnPrDF["PR"] = grnPrDF["PO Amount"] - grnPrDF["GRN"]

In [ ]:
grnPrDF

In [14]:
toSave = grnPrDF[["Cut/Veg", "Date", "GRN", "PR", "PO Amount"]]

In [ ]:
mail.logout()

In [15]:
toSave.index.name = "PO Number"

In [16]:
toSave.to_csv("GRN PR (Blinkit).csv")